In [124]:
import psycopg2
import psycopg2.extras
import pandas as pd
import random
from collections import defaultdict

In [92]:
conn = psycopg2.connect("dbname=dvdrental host='192.168.16.16' user='ags14' port=5430 password='bitnine123!'")
conn.autocommit = True
cursor = conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor)

In [93]:
cursor.execute("""
                  SELECT table_name
                  FROM information_schema.tables
                  WHERE table_schema = 'public'
                """)

In [94]:
tables = cursor.fetchall()
tables = [i['table_name'] for i in tables]
tables.remove('table_column_tbl_sh')
tables.remove('table_relation_sh')
tables.remove('pg_stat_statements_info')
tables.remove('pg_stat_statements')
tables.remove('table_column_info')
tables.remove('table_column_tbl')
tables.remove('table_relation')
print(len(tables))
tables

22


['actor',
 'actor_info',
 'customer_list',
 'film_list',
 'nicer_but_slower_film_list',
 'sales_by_film_category',
 'store',
 'sales_by_store',
 'staff_list',
 'address',
 'category',
 'city',
 'country',
 'customer',
 'film_actor',
 'film_category',
 'inventory',
 'language',
 'rental',
 'staff',
 'payment',
 'film']

In [95]:
for idx, i in enumerate(range(len(nodes))):
    nodes['table_name'].iloc[i]

In [96]:
table_n_cols = []
for i in tables:
    cursor.execute(f"""
                      SELECT table_name, column_name, data_type
                      FROM information_schema.columns
                      WHERE table_name = '{i}'
                    """)
    tmp = cursor.fetchall()
    for j in tmp:
        table_n_cols.append(j)
print(table_n_cols[0])
print(len(table_n_cols))

RealDictRow([('table_name', 'actor'), ('column_name', 'actor_id'), ('data_type', 'integer')])
128


In [129]:
import json

In [136]:
table_n_cols_converted = [dict(row) for row in table_n_cols]

In [142]:
with open("table_n_cols.json", "w", encoding='utf-8') as f:
    json.dump(table_n_cols, f)

In [144]:
with open("table_n_cols.json", 'r') as f:
    data = json.load(f)
data[:5]

[{'table_name': 'actor', 'column_name': 'actor_id', 'data_type': 'integer'},
 {'table_name': 'actor',
  'column_name': 'first_name',
  'data_type': 'character varying'},
 {'table_name': 'actor',
  'column_name': 'last_name',
  'data_type': 'character varying'},
 {'table_name': 'actor',
  'column_name': 'last_update',
  'data_type': 'timestamp without time zone'},
 {'table_name': 'actor_info',
  'column_name': 'actor_id',
  'data_type': 'integer'}]

In [126]:
table_n_cols_converted = [dict(row) for row in table_n_cols]
grouped_data = defaultdict(table_n_cols_converted)
for row in rows:
    grouped_data[row['table_name']].append(row['column_name'])

# Converting the defaultdict to the desired JSON format
result = [{table: columns} for table, columns in grouped_data.items()]
result

TypeError: first argument must be callable or None

In [119]:
table_n_cols[0]['table_name']

'actor'

In [ ]:
for i in table_n_cols:
    keys = i.keys()
    for key in keys:
        if i[key] in None:
            i[key] = i[key]
        else: 
            old = i[key]
            old.append(value)
        i[key] = values

In [97]:
df = pd.DataFrame(table_n_cols)
print(f"num of tables : {len(list(set(df['table_name'])))}")
print(f"num of columns : {len(list(set(df['column_name'])))}")
df

num of tables : 22
num of columns : 58


,table_name,column_name,data_type
0,actor,actor_id,integer
1,actor,first_name,character varying
2,actor,last_name,character varying
3,actor,last_update,timestamp without time zone
4,actor_info,actor_id,integer
...,...,...,...
123,film,replacement_cost,numeric
124,film,rating,USER-DEFINED
125,film,last_update,timestamp without time zone
126,film,special_features,ARRAY


In [ ]:
for i in range(len(df)):
    qwe

In [98]:
cols = df['column_name'].unique().tolist()
cols

['actor_id',
 'first_name',
 'last_name',
 'last_update',
 'film_info',
 'id',
 'name',
 'address',
 'zip code',
 'phone',
 'city',
 'country',
 'notes',
 'sid',
 'fid',
 'title',
 'description',
 'category',
 'price',
 'length',
 'rating',
 'actors',
 'total_sales',
 'store_id',
 'manager_staff_id',
 'address_id',
 'store',
 'manager',
 'address2',
 'district',
 'city_id',
 'postal_code',
 'category_id',
 'country_id',
 'customer_id',
 'email',
 'activebool',
 'create_date',
 'active',
 'film_id',
 'inventory_id',
 'language_id',
 'rental_id',
 'rental_date',
 'return_date',
 'staff_id',
 'username',
 'password',
 'picture',
 'payment_id',
 'amount',
 'payment_date',
 'release_year',
 'rental_duration',
 'rental_rate',
 'replacement_cost',
 'special_features',
 'fulltext']

In [99]:
len(cols)

58

- AGE 초기 설정

In [100]:
cursor.execute(f"""
                SET search_path TO ag_catalog;
                """)

- 레이블 생성

In [192]:
# cursor.execute(""" 
#                SELECT ag_catalog.create_vlabel('graph_v2', 'table');
#                """)

In [55]:
# cursor.execute(""" 
#                SELECT ag_catalog.create_vlabel('graph_v2', 'column');
#                """)

- 노드 생성

In [104]:
table_nodes = list(set(df['table_name']))

In [105]:
for i in table_nodes:
    cursor.execute(f""" 
                   SELECT * FROM cypher('graph_v2',$$
                   MERGE (:table {{table_name: '{i}'}}) 
                   $$) as (p agtype);
                   """)

In [106]:
for i in cols:
    cursor.execute(f""" 
                   SELECT * FROM cypher('graph_v2',$$
                   MERGE (:column {{column_name: '{i}'}}) 
                   $$) as (p agtype);
                   """)

- 엣지 생성(테이블-테이블)

In [107]:
cursor.execute("""SELECT tc.table_name,
                         ccu.column_name AS foreign_key_column,
                         ccu.table_name AS references_table
                         FROM information_schema.table_constraints tc
                         LEFT JOIN information_schema.constraint_column_usage ccu
                         ON tc.constraint_name = ccu.constraint_name
                         WHERE LOWER(tc.constraint_type) IN ('foreign key')
                         AND tc.table_name != 'ag_label'
                         """)

In [108]:
raw_rels = cursor.fetchall()
rels = pd.DataFrame(raw_rels)
rels

,table_name,foreign_key_column,references_table
0,customer,address_id,address
1,film_actor,actor_id,actor
2,film_actor,film_id,film
3,film_category,category_id,category
4,film_category,film_id,film
5,film,language_id,language
6,address,city_id,city
7,city,country_id,country
8,inventory,film_id,film
9,payment,customer_id,customer


In [109]:
# for i in range(len(rels)):
#     table_1 = rels.iloc[i]['table_name']
#     fkey = rels.iloc[i]['foreign_key_column']
#     table_2 = rels.iloc[i]['references_table']
#     cursor.execute(f""" 
#                    SELECT * from cypher('graph_v2', $$
#                    MATCH (v1:table), (v2:table)
#                    where v1.table_name = '{table_1}' and v2.table_name = '{table_2}'
#                    create (v1)-[e:ref {{fkey: '{fkey}'}}]->(v2)
#                    return e
#                    $$) as (e agtype);
#                    """)

In [110]:
for i in range(len(rels)):
    table_1 = rels.iloc[i]['table_name']
    fkey = rels.iloc[i]['foreign_key_column']
    table_2 = rels.iloc[i]['references_table']
    count = 0
    cursor.execute(f""" 
                   SELECT * from cypher('graph_v2', $$
                   MATCH (v1:table), (v2:table)
                   WHERE v1.table_name = '{table_1}' and v2.table_name = '{table_2}'
                   CREATE (v1)-[e:ref {{fkey: '{fkey}', count:{count} }}]->(v2)
                   RETURN e
                   $$) AS (e agtype);
                   """)

- 엣지 생성(테이블-컬럼)

In [111]:
df

,table_name,column_name,data_type
0,actor,actor_id,integer
1,actor,first_name,character varying
2,actor,last_name,character varying
3,actor,last_update,timestamp without time zone
4,actor_info,actor_id,integer
...,...,...,...
123,film,replacement_cost,numeric
124,film,rating,USER-DEFINED
125,film,last_update,timestamp without time zone
126,film,special_features,ARRAY


In [112]:
for i in range(len(df)):
    table_name = df.iloc[i]['table_name']
    column_name = df.iloc[i]['column_name']
    count = 0
    cursor.execute(f""" 
                   SELECT * from cypher('graph_v2', $$
                   MATCH (v1:table), (v2:column)
                   where v1.table_name = '{table_name}' and v2.column_name = '{column_name}'
                   create (v1)-[e:include {{ count: {count} }}]->(v2)
                   return e
                   $$) as (e agtype);
                   """)